### ④ 모델 활용

In [70]:
# 모듈 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [71]:
# 데이터 & 모델 불러오기
used = pd.read_csv('used_car_test.csv')
stats = pd.read_csv("used_stats.csv",index_col=0, encoding='CP949')	# 첫번째 열을 인덱스로 지정
used_model = tf.keras.models.load_model('used_model.h5')


In [72]:
#결측치 검출
used.isnull().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Power_bhp            0
dtype: int64

In [73]:
used_car = used  # 전처리용 데이터셋 생성

In [74]:
# 범주형 데이터에서 수치 분리
used_car['Mileage_kmpl'] = used_car['Mileage'].str.split(" ").str[0]
used_car['Engine_cc'] = used_car['Engine'].str.split(" ").str[0]
used_car['Power_bhp'] = used_car['Power'].str.split(" ").str[0]

# 실수형으로 변환
used_car['Mileage_kmpl'] = used_car['Mileage_kmpl'].astype(float)
used_car['Engine_cc'] = used_car['Engine_cc'].astype(float)
used_car['Power_bhp'] = used_car['Power_bhp'].astype(float)

In [75]:
# 불필요 행 삭제
used_car.drop(["Name"], axis=1,inplace=True)
used_car.drop(["Mileage"],axis=1,inplace=True)
used_car.drop(["Engine"],axis=1,inplace=True)
used_car.drop(["Power"],axis=1,inplace=True)
used_car.drop(["Location"],axis=1,inplace=True)

In [76]:
# 더미변수 생성
used_car = pd.get_dummies(used_car)

In [77]:
# 데이터셋 표준화
def norm(x):
  return (x - stats['mean']) / stats['std']		# x를 한행씩 읽어와서 표준화 후 리턴
normed = norm(used_car)			# 데이터셋 표준화



In [78]:
# 예측
pred = used_model.predict(normed)

In [79]:
# 원본 데이터셋에 예측열 생성
used["Price_predict"] = pred

In [80]:
# 원본 데이터와 예측값을 csv 파일로 저장
used.to_csv("used_car_predict.csv", index=False)